In [70]:
import re
import time
import tempfile, os
import random
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from selenium import webdriver
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [56]:
BASE_URL = 'https://www.nike.com/in/help'

driver = webdriver.Chrome()
driver.get(BASE_URL)
html = driver.page_source
driver.quit()


soup = BeautifulSoup(html, "html.parser")

for tag in soup.select("nav, header, footer, #nav-menu, .nav"):
    tag.decompose()

anchors = soup.find_all("a")

print(len(anchors))

for a in anchors:
    print(a.get("href"), a.get_text(strip=True))

25
/in/help GET HELP
/in/help/a/returns-policy What is Nike's return policy?
/in/help/a/how-to-return How do I return my Nike order?
/in/help/a/refund-info Where is my refund?
/in/help/a/returns-info-all View all
/in/help/a/shipping-delivery What are Nike's delivery options?
/in/help/a/international-shipping Can my Nike order be dispatched internationally?
/in/help/a/verify-passport Does my Nike order need to clear customs?
/in/help/a/shipping-delivery-info-all View all
/in/help/a/order-tracking Where is my Nike order?
/in/help/a/change-cancel-order Can I cancel or change my Nike order?
/in/help/a/payment-options What are Nike's payment options?
/in/help/a/orders-payment-info-all View all
/in/help/a/size-charts How do I find the right size and fit?
/in/help/a/product-discount How can I get Nike's best deals?
/in/help/a/product-advice Does Nike offer product advice?
/in/help/a/shopping-info-all View all
/in/help/a/member-benefits What is Nike Membership?
/in/help/a/nike-snkrs-draw How c

In [63]:
driver = webdriver.Chrome()

documents_list = []

for a in anchors[1:]:
    
    time.sleep(random.uniform(5, 10))
    
    driver.get(urljoin(BASE_URL,a.get('href')))
    
    WebDriverWait(driver, 20).until(
        EC.visibility_of_element_located((By.CSS_SELECTOR, "p.mb4-sm.responsive-body-2-1"))
    )
    WebDriverWait(driver, 20).until_not(
        EC.text_to_be_present_in_element(
            (By.CSS_SELECTOR, "body"), 
            "Building Preview"
        )
    )
    
    html = driver.page_source
    soup = BeautifulSoup(str(html), 'html.parser')
    for tag in soup.select("nav, header, footer, #nav-menu, .nav"):
        tag.decompose()

    text = soup.get_text(separator=' ')
    text = re.sub(r"\n\s*\n+", "\n", text)
    text = re.sub(r"[ \t]+", " ", text)
    text = text.strip()
    docs = Document(page_content=text)
    documents_list.append(docs)

driver.quit()

In [64]:
print(len(documents_list))
for doc in documents_list[:20]:
    print(doc.page_content)
    print("==="*10)

24
What is Nike's returns policy? | Nike Help 
 GET HELP What can we help you with? searchIcon What can we help you with? What is Nike's returns policy? Nike.com and Nike App purchases can be returned within 30 days ( some exceptions apply ). To be eligible to return, items must be in their original condition, unworn and unwashed with complete accessories, and the original packaging and tags must also be intact and in their original condition. This policy includes custom Nike By You sneakers. And remember, returns are always free for Nike Members—just make sure you sign in during checkout. 
 So go ahead, shop with confidence. Shop Nike Start a Return Join Us FAQs How do I return a Nike order? You can start a return online. For additional details, refer to our simple instructions . And remember, we always cover the return delivery costs for Nike Members . Can I exchange a Nike order? The best way to exchange an order is to simply place a new order for the item you want and return the it

In [65]:
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
splitted_docs = splitter.split_documents(documents_list[1:])
print(splitted_docs[:1])
print(len(splitted_docs))

[Document(metadata={}, page_content='How do I return my Nike order? | Nike Help')]
50


In [66]:
for doc in splitted_docs[:5]:
    print(doc.page_content)

How do I return my Nike order? | Nike Help
GET HELP What can we help you with? searchIcon What can we help you with? How do I return my Nike order? You can return unworn and unwashed Nike.com and Nike App orders within 30 days .* Simply follow the steps below to return your order. And remember, we'll cover the return delivery fees for Nike Members (guest orders will have a return fee subtracted from the refund). 1. Sign in to your Nike Member profile and go to your order history. Select "View or Manage" for the order you're returning, then select "Start Return". If you purchased as a guest, you'll need to access your order using your order number and email address. 2. Choose the items to return , provide the return reason from the dropdown menu, and select "Continue". 3. Submit your return and confirm your address —we'll automatically email a printable prepaid label. You can also choose to download the label. 4. Pack your items for delivery and securely attach your printed return deliv

In [67]:
from langchain_community.vectorstores import FAISS
from langchain_ollama.embeddings import OllamaEmbeddings

db = FAISS.from_documents(splitted_docs, OllamaEmbeddings(model='embeddinggemma:latest'))
retriever = db.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001FCEF4CDEE0>, search_kwargs={})

In [68]:
from langchain_core.tools import tool

@tool
def retriever_tool(query: str):
    """Search for information from Nike Help page."""
    docs = retriever.invoke(query)
    return "\n\n".join(d.page_content for d in docs)

In [ ]:
load_dotenv()

os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"

In [71]:
from langchain.agents import create_agent
from langchain_groq import ChatGroq

llm = ChatGroq(model='openai/gpt-oss-20b')
all_tools = [retriever_tool]
prompt = """
You are a tool-using agent. Follow these rules strictly:

1. If the user asks anything about Nike, returns, orders, delivery, refunds, or shoes → USE ONLY `retriever_tool`.
2. Never make up tool arguments.
3. Never call a tool twice for the same purpose.
4. After receiving tool results → answer clearly and concisely.

"""

agent = create_agent(model=llm, tools=all_tools, system_prompt=prompt)

In [ ]:
inputs = {"messages": [{"role": "user", "content": "What is Nike's returns policy for shoes?"}]}
response = agent.invoke(inputs, verbose=True)

In [81]:
AIMessage = response['messages'][3]
AIMessage.pretty_print()

================================== Ai Message ==================================

**Nike’s return policy for shoes**

| Time after purchase | Condition | What you get |
|---------------------|-----------|--------------|
| **≤ 30 days** | Unworn, unwashed shoes | Full refund (or exchange) – Nike covers return shipping for Nike Members; guests may pay a small fee. |
| **> 30 days – ≤ 2 years** | Defective or faulty shoes (material or workmanship fault) | Full refund (or replacement). Contact Nike to arrange return; you’ll receive a prepaid return label. |
| **Beyond 2 years** | No warranty – returns not accepted. |

**How to return**

1. Log in to your Nike Member profile (or use order number/email for guests).  
2. Go to **Order History → View/Manage → Start Return**.  
3. Select the shoes, choose a return reason, and submit.  
4. Print or download the prepaid return label and ship the package.  
5. If you bought from a retailer or an authorized store, return the shoes there instead.

*

In [83]:
inputs = {"messages": [{"role": "user", "content": "Tell me about the benefits of nike membership"}]}
response = agent.invoke(inputs, verbose=True)
AIMessage = response['messages'][3]
AIMessage.pretty_print()

================================== Ai Message ==================================

**Nike Membership benefits**

| Benefit | What you get |
|---------|--------------|
| **Free membership** | No cost to sign up |
| **Guidance from expert trainers** | Access to personalized training advice |
| **Member‑only experiences** | Exclusive events, both in‑person and online |
| **Special offers & deals** | Member‑only access to everyday discounts, milestone celebrations, and an annual birthday discount |
| **Insider info on drops** | Early or exclusive notifications about new releases |
| **Free delivery** | Eligible orders ship free |
| **Receiptless returns** | Return items via the Nike App or website using your order history |
| **Nike By You** | Personalise and order custom products |
| **Access to SNKRS, NRC, and NTC apps** | Use apps for drops, runs, and training |

Signing up is free, and you can explore these perks through the Nike App or website.


In [84]:
inputs = {"messages": [{"role": "user", "content": "Is there any paid nike membership"}]}
response = agent.invoke(inputs, verbose=True)
AIMessage = response['messages'][3]
AIMessage.pretty_print()

================================== Ai Message ==================================

Nike’s membership program is **free**. You can sign up at Nike.com or in the Nike App and enjoy member‑only perks—such as special offers, early access to drops, free delivery on eligible orders, and personalized training guidance—without paying any fee.
